# 建立LINE機器人

* 安裝套件

In [1]:
!pip install line-bot-sdk

## 使用官方範例程式

* 使用官方Github程式碼
https://github.com/line/line-bot-sdk-python

* 記得更改'YOUR_CHANNEL_ACCESS_TOKEN'、'YOUR_CHANNEL_SECRET'

In [2]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token='YOUR_CHANNEL_ACCESS_TOKEN')
handler = WebhookHandler('YOUR_CHANNEL_SECRET')


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text)]
            )
        )

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


## 安裝 Ngrok 進行跳板
* 註冊帳號 https://ngrok.com/
* 在 Your Authtoken 申請 token 並記下
* 讓colab可以跑 ngrok，先掛載 drive
* 下載ngrok並做好設定

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!mkdir -p /drive
#umount /drive
!mount --bind /content/drive/My\ Drive /drive
!mkdir -p /drive/ngrok-ssh
!mkdir -p ~/.ssh

# 切換到指定目錄
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh

# 下載並解壓最新版本的 Ngrok
!wget -c https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz

# 複製 ngrok 可執行文件並給予執行權限
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok


Mounted at /content/drive
mkdir: cannot stat ‘/drive’: Transport endpoint is not connected
/drive/ngrok-ssh
--2024-08-21 14:28:26--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 54.237.133.81, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8974299 (8.6M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   8.56M  11.6MB/s    in 0.7s    

2024-08-21 14:28:27 (11.6 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [8974299/8974299]

ngrok


* 將 token 改為自己的

In [4]:
!./ngrok authtoken 4oVAUwYgzt1HsxSoVjds2_5Tz8dzMu6xDF4hyGYbqap

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


* 安裝 ngrok、LINE 相依套件

In [5]:
!pip install pyngrok

## 重新執行API程式
* 注意每次執行的URL會不一樣

In [9]:
from flask import Flask, request, abort
from pyngrok import ngrok
import os

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import Configuration, ApiClient, MessagingApi, ReplyMessageRequest, TextMessage
from linebot.v3.webhooks import MessageEvent, TextMessageContent

app = Flask(__name__)

# 設定你的 Channel Access Token 和 Channel Secret
configuration = Configuration(access_token='YOUR_CHANNEL_ACCESS_TOKEN')
handler = WebhookHandler('YOUR_CHANNEL_SECRET')

@app.route("/callback", methods=['POST'])
def callback():
    # 獲取 X-Line-Signature header 值
    signature = request.headers['X-Line-Signature']

    # 獲取文本內容
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # 處理 webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text)]
            )
        )

if __name__ == "__main__":
    port = 5000
    public_url = ngrok.connect(port).public_url
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")

    # 更新 LINE webhook URL
    os.environ["LINE_WEBHOOK_URL"] = public_url + "/callback"
    print(f"Webhook URL: {os.environ['LINE_WEBHOOK_URL']}")
    app.run(port=port)

 * ngrok tunnel "https://a8e4-35-224-254-65.ngrok-free.app" -> "http://127.0.0.1:5000"
Webhook URL: https://a8e4-35-224-254-65.ngrok-free.app/callback
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 14:38:27] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 14:46:08] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 14:48:05] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 14:48:08] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 14:48:10] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Aug/2024 14:48:11] "POST /callback HTTP/1.1" 200 -


# 使用GCP建立機器人

## Cloud run 函數
* echo機器人

In [ ]:
from flask import abort, jsonify
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import Configuration, ApiClient, MessagingApi, ReplyMessageRequest, TextMessage
from linebot.v3.webhooks import MessageEvent, TextMessageContent

# 設定你的 Channel Access Token 和 Channel Secret
configuration = Configuration(access_token='YOUR_CHANNEL_ACCESS_TOKEN')
handler = WebhookHandler('YOUR_CHANNEL_SECRET')

def line_bot(request):
    # 獲取 X-Line-Signature header 值
    signature = request.headers.get('X-Line-Signature', '')

    # 獲取文本內容
    body = request.get_data(as_text=True)
    print(f"Request body: {body}")

    # 處理 webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text)]
            )
        )

## Gemini API 機器人

In [ ]:
import os
from flask import abort
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import Configuration, ApiClient, MessagingApi, ReplyMessageRequest, TextMessage
from linebot.v3.webhooks import MessageEvent, TextMessageContent
import google.generativeai as genai

# 設定你的 Channel Access Token 和 Channel Secret
configuration = Configuration(access_token='YOUR_CHANNEL_ACCESS_TOKEN')
handler = WebhookHandler('YOUR_CHANNEL_SECRET')

# 設置 GCP API 金鑰
GEMINI_API_KEY = 'YOUR_API_KEY'

# 配置 Gemini
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-pro')

def generate_gemini_response(prompt):
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error generating Gemini response: {str(e)}")
        return "很抱歉我無法回答，請稍後再試。"

def line_bot_webhook(request):
    # 獲取 X-Line-Signature header 值
    signature = request.headers.get('X-Line-Signature', '')

    # 獲取文本內容
    body = request.get_data(as_text=True)
    print(f"Request body: {body}")

    # 處理 webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    user_message = event.message.text
    gemini_response = generate_gemini_response(user_message)

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=gemini_response)]
            )
        )

# 進入點
def main(request):
    return line_bot_webhook(request)